# Cartpole Dengeleme için RL Eğitimi

Bu not defteri, [AI for Beginners Müfredatı](http://aka.ms/ai-beginners) kapsamında yer almaktadır. [Resmi PyTorch eğitimi](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) ve [bu Cartpole PyTorch uygulamasından](https://github.com/yc930401/Actor-Critic-pytorch) ilham alınarak hazırlanmıştır.

Bu örnekte, bir modelin yatay bir düzlemde sağa ve sola hareket edebilen bir arabada bir direği dengelemesini sağlamak için RL kullanacağız. Direği simüle etmek için [OpenAI Gym](https://www.gymlibrary.ml/) ortamını kullanacağız.

> **Not**: Bu dersin kodunu yerel olarak (örneğin, Visual Studio Code'dan) çalıştırabilirsiniz; bu durumda simülasyon yeni bir pencerede açılacaktır. Kodu çevrimiçi çalıştırırken, [burada](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) açıklandığı gibi kodda bazı değişiklikler yapmanız gerekebilir.

Gym'in yüklü olduğundan emin olarak başlayacağız:


In [ ]:
import sys
!{sys.executable} -m pip install gym

Şimdi CartPole ortamını oluşturalım ve üzerinde nasıl işlem yapacağımızı görelim. Bir ortamın şu özellikleri vardır:

* **Eylem alanı**, simülasyonun her adımında gerçekleştirebileceğimiz olası eylemlerin kümesidir  
* **Gözlem alanı**, yapabileceğimiz gözlemlerin alanıdır  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

Simülasyonun nasıl çalıştığını görelim. Aşağıdaki döngü, `env.step` sonlandırma bayrağı `done` döndürmediği sürece simülasyonu çalıştırır. Eylemleri rastgele seçmek için `env.action_space.sample()` kullanacağız, bu da deneyin muhtemelen çok hızlı bir şekilde başarısız olacağı anlamına gelir (CartPole ortamı, CartPole'un hızı, konumu veya açısı belirli sınırların dışına çıktığında sonlanır).

> Simülasyon yeni bir pencerede açılacaktır. Kodu birkaç kez çalıştırabilir ve nasıl davrandığını görebilirsiniz.


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

Gözlemlerin 4 sayı içerdiğini fark edebilirsiniz. Bunlar şunlardır:
- Arabanın konumu
- Arabanın hızı
- Direğin açısı
- Direğin dönüş hızı

`rew`, her adımda aldığımız ödüldür. CartPole ortamında, her simülasyon adımı için 1 puan ödül alırsınız ve amaç toplam ödülü maksimize etmektir, yani CartPole'un düşmeden dengede kalabildiği süreyi artırmaktır.

Pekiştirmeli öğrenme sırasında, amacımız her durum $s$ için hangi eylemi $a$ yapmamız gerektiğini söyleyen bir **politika** $\pi$ eğitmek, yani temelde $a = \pi(s)$.

Eğer olasılıksal bir çözüm istiyorsanız, politikayı her eylem için bir olasılık seti döndüren bir şey olarak düşünebilirsiniz, yani $\pi(a|s)$, durum $s$'de eylem $a$'yı yapmamız gerektiği olasılığı anlamına gelir.

## Politika Gradyan Yöntemi

En basit RL algoritmasında, **Politika Gradyanı** olarak adlandırılan yöntemde, bir sinir ağı eğiterek bir sonraki eylemi tahmin edeceğiz.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

Ağı eğitmek için birçok deney yaparak ve her çalıştırmadan sonra ağımızı güncelleyerek ilerleyeceğiz. Deneyi çalıştıracak ve sonuçları (sözde **iz**) - tüm durumlar, eylemler (ve önerilen olasılıkları) ve ödülleri döndürecek bir fonksiyon tanımlayalım:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

Eğitilmemiş ağ ile bir bölüm çalıştırabilir ve toplam ödülün (diğer bir deyişle bölümün uzunluğu) çok düşük olduğunu gözlemleyebilirsiniz:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

Politika gradyan algoritmasının zorlu yönlerinden biri **indirimli ödüller** kullanmaktır. Fikir, oyunun her adımında toplam ödüller vektörünü hesaplamak ve bu süreçte erken ödülleri bir $gamma$ katsayısı kullanarak indirimli hale getirmektir. Ayrıca, elde edilen vektörü normalize ederiz çünkü bunu eğitimimizi etkilemek için ağırlık olarak kullanacağız:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

Şimdi gerçek eğitime başlayalım! 300 bölüm çalıştıracağız ve her bölümde aşağıdaki adımları gerçekleştireceğiz:

1. Deneyi çalıştır ve izleri topla
1. Alınan eylemler ile tahmin edilen olasılıklar arasındaki farkı (`gradients`) hesapla. Fark ne kadar az olursa, doğru eylemi seçtiğimizden o kadar emin oluruz.
1. İndirimli ödülleri hesapla ve `gradients` ile çarp - bu, daha yüksek ödüllere sahip adımların nihai sonuç üzerinde daha fazla etkisi olmasını, düşük ödüllü adımların ise daha az etkisi olmasını sağlar.
1. Sinir ağımız için beklenen hedef eylemler, kısmen çalıştırma sırasında tahmin edilen olasılıklardan, kısmen de hesaplanan `gradients` değerlerinden alınacaktır. `alpha` parametresini kullanarak `gradients` ve ödüllerin ne ölçüde dikkate alınacağını belirleyeceğiz - bu, pekiştirme algoritmasının *öğrenme oranı* olarak adlandırılır.
1. Son olarak, ağımızı durumlar ve beklenen eylemler üzerinde eğitiyoruz ve süreci tekrarlıyoruz.


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

Şimdi sonucu görmek için bölümü render ederek çalıştıralım:


In [ ]:
_ = run_episode(render=True)

Umarım artık çubuğun oldukça iyi dengede durabildiğini görebiliyorsunuz!

## Aktör-Kritik Modeli

Aktör-Kritik modeli, politika gradyanlarının daha ileri bir geliştirmesidir. Bu modelde, hem politikayı hem de tahmini ödülleri öğrenmek için bir sinir ağı oluştururuz. Ağın iki çıktısı olacaktır (ya da bunu iki ayrı ağ olarak düşünebilirsiniz):
* **Aktör**, politika gradyan modelinde olduğu gibi, durum olasılık dağılımını vererek hangi eylemin yapılması gerektiğini önerecektir.
* **Kritik**, bu eylemlerden elde edilecek ödüllerin ne olacağını tahmin eder. Verilen durumda gelecekteki toplam tahmini ödülleri döndürür.

Şimdi böyle bir modeli tanımlayalım:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

`discounted_rewards` ve `run_episode` fonksiyonlarımızı biraz değiştirmemiz gerekecek:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


Şimdi ana eğitim döngüsünü çalıştıracağız. Uygun kayıp fonksiyonlarını hesaplayarak ve ağ parametrelerini güncelleyerek manuel ağ eğitim sürecini kullanacağız:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## Özet

Bu demoda iki RL algoritması gördük: basit politika gradyanı ve daha sofistike aktör-eleştirmen. Bu algoritmaların durum, eylem ve ödül gibi soyut kavramlarla çalıştığını görebilirsiniz - bu nedenle çok farklı ortamlara uygulanabilirler.

Pekiştirmeli öğrenme, yalnızca nihai ödüle bakarak problemi çözmek için en iyi stratejiyi öğrenmemizi sağlar. Etiketlenmiş veri kümelerine ihtiyaç duymamamız, modellerimizi optimize etmek için simülasyonları birçok kez tekrarlamamıza olanak tanır. Ancak, RL'de hâlâ birçok zorluk bulunmaktadır ve bu ilginç AI alanına daha fazla odaklanmaya karar verirseniz bunları öğrenebilirsiniz.



---

**Feragatname**:  
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluğu sağlamak için çaba göstersek de, otomatik çevirilerin hata veya yanlışlık içerebileceğini lütfen unutmayın. Belgenin orijinal dili, yetkili kaynak olarak kabul edilmelidir. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından kaynaklanan yanlış anlamalar veya yanlış yorumlamalar için sorumluluk kabul edilmez.
